In [1]:
import sys
import json
sys.path.append('..')
import pandas as pd
from src.feature_engineering import FeatureCreator

In [2]:
df = pd.read_csv('../data/raw/Suicide_Detection.csv', usecols=['text','class'])
df['class'] = df['class'].map({'suicide': 1, 'non-suicide':0})

In [3]:
with open('../config/config.json', 'r') as file:
        config = json.load(file)

procedure_feature_preclean = config['feature_engineering']['procedure_before_preprocessing']
procedure_feature_postclean = config['feature_engineering']['procedure_after_preprocessing']

In [4]:
feature_creator_preclean = FeatureCreator(procedure=procedure_feature_preclean)
length_exclamation = [feature_creator_preclean.fit(document) for document in df['text']]
length = [x for x,y in length_exclamation]
exclamation = [y for x,y in length_exclamation]

In [5]:
df = pd.read_csv('../data/prepared/prepared.csv', index_col=0)
df['corpus'] = df['corpus'].apply(lambda x: x[1:-1].replace("'", "").split(', '))

In [6]:
feature_creator_postclean = FeatureCreator(procedure=procedure_feature_postclean)
feature_creator_postclean.fit_vectorizer(df['corpus'])

print('Vectorizer ready!')
sentiment_keyword = [feature_creator_postclean.fit(document) for document in df['corpus']]
sentiment = [x for x,y in sentiment_keyword]
keyword = [y for x,y in sentiment_keyword]

Vectorizer ready!


In [7]:
sentiment = [x for x,y in sentiment_keyword]
keyword = [y for x,y in sentiment_keyword]

In [8]:
length = pd.DataFrame(length)
exclamation = pd.DataFrame(exclamation)

length = length[length.index.isin(df.index)]
exclamation = exclamation[exclamation.index.isin(df.index)]

In [9]:
df_feature = pd.concat(
    [length,
    exclamation,
    pd.DataFrame(sentiment, index=df.index),
    pd.DataFrame(keyword, index=df.index),
    df['class'],
    ],
    axis=1
)
df_feature.columns = ['length', 'exclamation', 'negative', 'neutral', 'positive', 'compound', 'keyword', 'class']
df_feature.index = df.index

In [10]:
df_feature.to_csv("../data/prepared/new_features.csv")